In [ ]:
# This takes photometry of stars through apertures arranged on a grid which moves
# to compensate for poor telescope guiding

# Created 2021 Aug. 20 by E.S.

In [5]:
import pandas as pd
import numpy as np
#from astropy.stats import sigma_clipped_stats
#from photutils.detection import DAOStarFinder
from photutils.aperture import CircularAperture, aperture_photometry
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import matplotlib.pyplot as plt
from astropy.io import fits, ascii
import glob
import os

%matplotlib qt

In [6]:
stem_data = "/Users/bandari/Documents/git.repos/kriz_projects/data/2021-08-04/00_dark_subtracted_flat_fielded/"
stem_star_positions = "/Users/bandari/Documents/git.repos/kriz_projects/notebooks_for_development/output_other/"

In [7]:
# names of files

file_names_stars = sorted(glob.glob(stem_data + "V1432_Aql-*fit"))

file_names_pos = sorted(glob.glob(stem_star_positions + "V1432_Aql-*dat"))

# file of measured displacements between images
file_name_disp = "displacements_v1432_aql.csv"

In [8]:
# read in image displacements
disp_images = pd.read_csv(stem_star_positions+file_name_disp)
# read in first image's star locations (we will shift this)
initial_star_pos = pd.read_csv(file_names_pos[0], delim_whitespace=True)

## ## START FOR-LOOP

for num_star_file in range(0,len(file_names_stars)):

    # read in star image
    star_image = fits.open(file_names_stars[num_star_file])[0].data

    # retrieve displacement
    idx = disp_images["file_name"]==str(os.path.basename(file_names_stars[num_star_file]))
    x_disp = disp_images["x_off_pix"].where(idx).dropna().values[0]
    y_disp = disp_images["y_off_pix"].where(idx).dropna().values[0]

    # shift grid of first (i.e., [0]) array of images by this much
    positions = np.transpose((np.subtract(initial_star_pos['xcentroid'],x_disp), 
                              np.subtract(initial_star_pos['ycentroid'],y_disp)))
    apertures = CircularAperture(positions, r=4.)

    phot_table = aperture_photometry(star_image, apertures)
    phot_table['aperture_sum'].info.format = '%.8g'  # for consistent table output
    print(phot_table)

    # write text file with aperture photometry
    text_file_name = "output_other/aper_photom_"+str(os.path.basename(file_names_stars[num_star_file])).split(".")[-2]+".dat"
    ascii.write(phot_table, text_file_name, overwrite=True)
    print("Wrote out "+text_file_name)

    '''
    # write FYI plot
    norm = ImageNormalize(stretch=SqrtStretch())
    plt.imshow(star_image, cmap='Greys', origin='lower', norm=norm,
               interpolation='nearest')
    apertures.plot(color='blue', lw=1.5, alpha=0.5)
    '''

## ## END FOR-LOOP


 id      xcenter            ycenter       aperture_sum
           pix                pix                     
--- ------------------ ------------------ ------------
  1          350.24465 1.1349221999999999    136059.04
  2         0.18494507  4.425652599999999    195530.14
  3          359.27633 3.2864478999999998    176732.65
  4          481.88226          4.4149629    178869.54
  5          8.3225035          9.3288388    181146.45
  6          16.730306          9.4884733    177736.58
  7          162.23037          8.8991306    178339.09
  8           387.8473  9.892791599999999    180027.76
  9  77.49881500000001          10.993477    234339.53
 10 24.935910999999997           12.60379    198518.83
...                ...                ...          ...
293          8.0940882          502.78714    176884.05
294           328.6585 502.98010999999997    177199.31
295          165.53355 504.19599000000005    180884.78
296          106.77722          506.85355    177524.61
297 211.97

 id       xcenter             ycenter       aperture_sum
            pix                 pix                     
--- -------------------- ------------------ ------------
  1            349.84465 0.6849221999999999    123341.54
  2 -0.21505493000000003  3.975652599999999    157077.86
  3            358.87633 2.8364478999999996    170387.48
  4            481.48226 3.9649628999999997    178177.56
  5   7.9225034999999995          8.8788388     180042.2
  6            16.330306  9.038473300000001    176855.75
  7            161.83037          8.4491306    177320.11
  8   387.44730000000004          9.4427916    178144.16
  9            77.098815 10.543477000000001    233886.69
 10            24.535911           12.15379    197553.88
...                  ...                ...          ...
293   7.6940881999999995 502.33714000000003    176136.84
294             328.2585          502.53011    176393.58
295   165.13354999999999 503.74599000000006    179766.15
296            106.37722       

Wrote out output_other/aper_photom_V1432_Aql-0014.dat
 id      xcenter            ycenter       aperture_sum
           pix                pix                     
--- ------------------ ------------------ ------------
  1 349.32464999999996         -0.4950778    89261.171
  2        -0.73505493 2.7956525999999995    102216.51
  3 358.35632999999996 1.6564478999999999    145632.45
  4 480.96225999999996          2.7849629    169518.88
  5          7.4025035          7.6988388    179793.22
  6 15.810305999999999  7.858473300000001    176876.44
  7          161.31037 7.2691305999999996    176685.02
  8           386.9273          8.2627916    178414.55
  9          76.578815           9.363477    233870.15
 10 24.015910999999996 10.973790000000001    197153.02
...                ...                ...          ...
293          7.1740882          501.15714    175596.61
294           327.7385          501.35011    175783.76
295          164.61355 502.56599000000006    179234.85
296        

 id       xcenter             ycenter       aperture_sum
            pix                 pix                     
--- ------------------- ------------------- ------------
  1           349.22465 -0.6450778000000001    84900.762
  2 -0.8350549300000001   2.645652599999999     93977.98
  3           358.25633  1.5064478999999997    141673.53
  4           480.86226  2.6349628999999997    166551.76
  5           7.3025035  7.5488387999999995    178779.56
  6           15.710306  7.7084733000000005    176079.68
  7  161.21036999999998   7.119130599999999    176108.93
  8  386.82730000000004           8.1127916    177908.01
  9   76.47881500000001   9.213477000000001    232059.27
 10  23.915910999999998            10.82379    197000.58
...                 ...                 ...          ...
293           7.0740882  501.00714000000005    175098.91
294            327.6385           501.20011    175449.35
295  164.51354999999998   502.4159900000001    178774.93
296           105.75722        

 id      xcenter             ycenter       aperture_sum
           pix                 pix                     
--- ------------------ ------------------- ------------
  1          349.05465 -0.7050778000000002    82731.713
  2        -1.00505493  2.5856525999999995    85229.127
  3          358.08633  1.4464478999999997    140212.86
  4          480.69226           2.5749629    165328.46
  5          7.1325035           7.4888388    178712.81
  6          15.540306   7.648473300000001    175842.22
  7          161.04037           7.0591306    175634.73
  8           386.6573   8.052791599999999    177735.45
  9  76.30881500000001            9.153477    232220.75
 10 23.745910999999996            10.76379    196760.99
...                ...                 ...          ...
293          6.9040882  500.94714000000005    174984.12
294           327.4685           501.14011    174926.82
295          164.34355   502.3559900000001    177999.09
296          105.58722           505.01355    17

...

Wrote out output_other/aper_photom_V1432_Aql-2273.dat
 id       xcenter            ycenter       aperture_sum
            pix                pix                     
--- ------------------- ------------------ ------------
  1           259.85465         14.5149222    205373.23
  2        -90.20505493 17.805652600000002          nan
  3           268.88633         16.6664479    203536.42
  4           391.49226         17.7949629    204000.28
  5         -82.0674965 22.708838800000002          nan
  6          -73.659694         22.8684733          nan
  7            71.84037 22.279130600000002    206113.19
  8  297.45730000000003 23.272791599999998    205279.02
  9 -12.891184999999993          24.373477          nan
 10  -65.45408900000001           25.98379          nan
...                 ...                ...          ...
293         -82.2959118          516.16714          nan
294  238.26850000000002          516.36011          nan
295   75.14354999999999          517.57599        

 id       xcenter            ycenter       aperture_sum
            pix                pix                     
--- ------------------- ------------------ ------------
  1           259.85465         14.5149222    204741.36
  2        -90.20505493 17.805652600000002          nan
  3           268.88633         16.6664479    203721.44
  4           391.49226         17.7949629    204620.42
  5         -82.0674965 22.708838800000002          nan
  6          -73.659694         22.8684733          nan
  7            71.84037 22.279130600000002    205392.11
  8  297.45730000000003 23.272791599999998    204802.23
  9 -12.891184999999993          24.373477          nan
 10  -65.45408900000001           25.98379          nan
...                 ...                ...          ...
293         -82.2959118          516.16714          nan
294  238.26850000000002          516.36011          nan
295   75.14354999999999          517.57599          nan
296            16.38722          520.23355      

Wrote out output_other/aper_photom_V1432_Aql-2286.dat
 id       xcenter            ycenter       aperture_sum
            pix                pix                     
--- ------------------- ------------------ ------------
  1           260.84465         15.1149222    205518.32
  2  -89.21505493000001         18.4056526          nan
  3  269.87632999999994         17.2664479    204079.55
  4           392.48226         18.3949629    204631.73
  5  -81.07749650000001         23.3088388          nan
  6          -72.669694         23.4684733          nan
  7   72.83036999999999         22.8791306    205740.47
  8  298.44730000000004         23.8727916    205075.66
  9 -11.901184999999998 24.973477000000003          nan
 10          -64.464089           26.58379          nan
...                 ...                ...          ...
293         -81.3059118          516.76714          nan
294            239.2585          516.96011          nan
295   76.13354999999999  518.1759900000001        

Wrote out output_other/aper_photom_V1432_Aql-2292.dat
 id       xcenter            ycenter       aperture_sum
            pix                pix                     
--- ------------------- ------------------ ------------
  1  260.32464999999996         15.0649222    207634.88
  2        -89.73505493         18.3556526          nan
  3  269.35632999999996         17.2164479    206191.86
  4  391.96225999999996         18.3449629    206414.05
  5         -81.5974965         23.2588388          nan
  6          -73.189694 23.418473300000002          nan
  7   72.31036999999999         22.8291306    207937.57
  8            297.9273         23.8227916    206762.53
  9 -12.421184999999994          24.923477          nan
 10  -64.98408900000001           26.53379          nan
...                 ...                ...          ...
293         -81.8259118          516.71714          nan
294            238.7385  516.9101099999999          nan
295   75.61354999999999          518.12599        

Wrote out output_other/aper_photom_V1432_Aql-2298.dat
 id      xcenter            ycenter       aperture_sum
           pix                pix                     
--- ------------------ ------------------ ------------
  1          260.06465         14.3649222    208194.82
  2 -89.99505493000001         17.6556526          nan
  3 269.09632999999997         16.5164479     207028.3
  4 391.70225999999997         17.6449629    207936.33
  5 -81.85749650000001         22.5588388          nan
  6 -73.44969400000001         22.7184733          nan
  7  72.05036999999999         22.1291306    207885.17
  8           297.6673         23.1227916    207584.07
  9         -12.681185 24.223477000000003          nan
 10         -65.244089           25.83379          nan
...                ...                ...          ...
293        -82.0859118          516.01714          nan
294           238.4785          516.21011          nan
295  75.35354999999998  517.4259900000001          nan
296 16.5972

In [42]:
'''
# for plotting
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=4.)
norm = ImageNormalize(stretch=SqrtStretch())
plt.imshow(data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
apertures.plot(color='blue', lw=1.5, alpha=0.5)
'''

In [44]:
# star images
file_names_stars = sorted(glob.glob(stem_data + "V1432_Aql-*fit"))

# darks
file_names_darks = sorted(glob.glob(stem_data + "d*fit"))

# flats
file_names_flats = sorted(glob.glob(stem_data + "flat*fit"))